In [1]:
from embedding_reader import EmbeddingReader
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import os
import math

# Load LAION CLIP Data

### Fetch if not already stored

In [2]:
# emebeddings are .npy
# metadata is .paraquet
embedding_reader = EmbeddingReader(
    embeddings_folder="https://mystic.the-eye.eu/public/AI/cah/laion5b/embeddings/laion2B-en/",
    metadata_folder="https://mystic.the-eye.eu/public/AI/cah/laion5b/metadata/laion2B-en/",
    meta_columns=['SAMPLE_ID', 'TEXT'],
    file_format="parquet_npy"
)
print("embedding count", embedding_reader.count)
print("dimension", embedding_reader.dimension)
print("total size", embedding_reader.total_size)
print("byte per item", embedding_reader.byte_per_item)

100%|██████████████████████████████████████████████████████████████| 4611/4611 [03:19<00:00, 23.07it/s]

embedding count 116341562
dimension 768
total size 178700639232
byte per item 1536


In [3]:
for emb, meta in embedding_reader(batch_size=10 ** 4, start=0, end=10 ** 4, show_progress=True):
    print(emb.shape)
    print(meta.size)

100%|███████████████████████████████████████████████████████████████████| 1/1 [02:06<00:00, 126.01s/it]

(10000, 768)
30000


In [6]:
meta['emb'] = emb.tolist()

In [15]:
meta.to_parquet('laion2B-10000.parquet.gzip', compression='gzip')

### Create Dataset and Splits

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
torch.cuda.set_device(3)

In [4]:
meta = pd.read_parquet('laion2B-10000.parquet.gzip')

In [5]:
emb = torch.Tensor(meta['emb'])

/tmp/ipykernel_11020/3476993897.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  emb = torch.Tensor(meta['emb'])


In [6]:
train_idx = int(len(meta) * 0.8)
x_train, y_train = meta['TEXT'][:train_idx], emb[:train_idx]
x_test, y_test = meta['TEXT'][train_idx:], emb[train_idx:]

In [7]:
x_train.shape, y_train.size(), x_test.shape, y_test.size()

((8000,), torch.Size([8000, 768]), (2000,), torch.Size([2000, 768]))

# Fine-tune LM -> Predict CLIP image embeddings

In [8]:
from transformers import AutoModel, AutoTokenizer

In [9]:
epochs = 4
batch_size = 16
train_batches = math.ceil(x_train.shape[0]/batch_size)
test_batches = math.ceil(x_test.shape[0]/batch_size)

## BERT

In [10]:
class CLIPEmbBERT(nn.Module):
    def __init__(self):
        super(CLIPEmbBERT, self).__init__()
        self.model = AutoModel.from_pretrained("bert-base-cased")
          ### New layers: None

    def forward(self, tokens, mask):
        cls_emebdding = self.model(tokens, attention_mask=mask).pooler_output

        return cls_emebdding 

In [13]:
bert_model = CLIPEmbBERT()
bert_model.to(device)

bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(bert_model.parameters(), lr=5e-3, weight_decay=1e-5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
x_train_bert = bert_tokenizer.batch_encode_plus(list(x_train), return_tensors='pt', padding=True, add_special_tokens=True).to(device)
y_train = y_train.to(device)

In [15]:
%%time
bert_model.train()
for epoch in range(epochs):
    
    rand_ids = torch.randperm(x_train_bert['input_ids'].size()[0])
    X = x_train_bert['input_ids'][rand_ids]
    masks = x_train_bert['attention_mask'][rand_ids]
    
    total_loss = 0.0
    
    for i in tqdm(range(0, X.size()[0], batch_size)):
        optimizer.zero_grad()
        
        outputs = bert_model(X[i:i+batch_size], mask=masks[i:i+batch_size])
        
        loss = torch.sqrt(criterion(outputs, y_train[i:i+batch_size]))
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    print(f"Loss Epoch {epoch+1} / {epochs}: {total_loss/train_batches}")

100%|█████████████████████████████████████████████| 500/500 [01:49<00:00,  4.59it/s]


Loss Epoch 1 / 4: 0.12889956278726458


100%|█████████████████████████████████████████████| 500/500 [01:48<00:00,  4.60it/s]


Loss Epoch 2 / 4: 0.0293963553160429


100%|█████████████████████████████████████████████| 500/500 [01:48<00:00,  4.60it/s]


Loss Epoch 3 / 4: 0.02884599167108536


100%|█████████████████████████████████████████████| 500/500 [01:48<00:00,  4.60it/s]

Loss Epoch 4 / 4: 0.02861583412066102
CPU times: user 7min 5s, sys: 5.34 s, total: 7min 11s
Wall time: 7min 15s


In [87]:
x_test_bert = bert_tokenizer.batch_encode_plus(list(x_test), return_tensors='pt', padding=True, add_special_tokens=True).to(device)
y_test = y_test.to(device)

In [92]:
predicted_embs = np.array([[0]*768])
bert_model.eval()
with torch.no_grad():
    X = x_test_bert['input_ids']
    masks = x_test_bert['attention_mask']
    total_loss = 0.0
    for i in tqdm(range(0, X.size()[0], batch_size)):
        outputs = bert_model(X[i:i+batch_size], mask=masks[i:i+batch_size])
        predicted_embs = np.concatenate([predicted_embs, outputs.cpu()])

        loss = torch.sqrt(criterion(outputs, y_test[i:i+batch_size]))
        total_loss += loss.item()

total_loss/test_batches

100%|█████████████████████████████████████████████| 125/125 [00:09<00:00, 13.81it/s]


0.02841430501639843

### Save embeddings

In [95]:
np.save("BERT_test_preds.npy", predicted_embs[1:])
np.save("LAION_test_gt.npy", np.array(y_test.cpu()))

## DistilBERT

In [10]:
from transformers import DistilBertForSequenceClassification
class CLIPEmbDistilBERT(nn.Module):
    def __init__(self):
        super(CLIPEmbDistilBERT, self).__init__()
        self.model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=768)
          ### New layers: None

    def forward(self, tokens, mask):
        cls_emebdding = self.model(tokens, attention_mask=mask).logits

        return cls_emebdding

In [11]:
distilbert_model = CLIPEmbDistilBERT()
distilbert_model.to(device)

distilbert_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(distilbert_model.parameters(), lr=3e-3, weight_decay=1e-5)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [12]:
x_train_distilbert = distilbert_tokenizer.batch_encode_plus(list(x_train), return_tensors='pt', padding=True, add_special_tokens=True).to(device)
y_train = y_train.to(device)

In [13]:
distilbert_model.train()
for epoch in range(epochs):
    
    rand_ids = torch.randperm(x_train_distilbert['input_ids'].size()[0])
    X = x_train_distilbert['input_ids'][rand_ids]
    masks = x_train_distilbert['attention_mask'][rand_ids]
    
    total_loss = 0.0
    
    for i in tqdm(range(0, X.size()[0], batch_size)):
        optimizer.zero_grad()
        
        outputs = distilbert_model(X[i:i+batch_size], mask=masks[i:i+batch_size])
        
        loss = torch.sqrt(criterion(outputs, y_train[i:i+batch_size]))
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    print(f"Loss Epoch {epoch+1} / {epochs}: {total_loss/train_batches}")

100%|█████████████████████████████████████████████| 500/500 [00:57<00:00,  8.71it/s]


Loss Epoch 1 / 4: 0.0430491885766387


100%|█████████████████████████████████████████████| 500/500 [00:57<00:00,  8.66it/s]


Loss Epoch 2 / 4: 0.02832267015427351


100%|█████████████████████████████████████████████| 500/500 [00:57<00:00,  8.75it/s]


Loss Epoch 3 / 4: 0.028338031571358443


100%|█████████████████████████████████████████████| 500/500 [00:56<00:00,  8.92it/s]

Loss Epoch 4 / 4: 0.02832708839699626


In [14]:
x_test_distilbert = distilbert_tokenizer.batch_encode_plus(list(x_test), return_tensors='pt', padding=True, add_special_tokens=True).to(device)
y_test = y_test.to(device)

In [15]:
predicted_embs = np.array([[0]*768])
distilbert_model.eval()
with torch.no_grad():
    X = x_test_distilbert['input_ids']
    masks = x_test_distilbert['attention_mask']
    total_loss = 0.0
    for i in tqdm(range(0, X.size()[0], batch_size)):
        outputs = distilbert_model(X[i:i+batch_size], mask=masks[i:i+batch_size])
        predicted_embs = np.concatenate([predicted_embs, outputs.cpu()])

        loss = torch.sqrt(criterion(outputs, y_test[i:i+batch_size]))
        total_loss += loss.item()

total_loss/test_batches

100%|█████████████████████████████████████████████| 125/125 [00:04<00:00, 25.31it/s]


0.028231130450963975

### Save embeddings

In [16]:
np.save("DistilBERT_test_preds.npy", predicted_embs[1:])